In [1]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'\-')
lg.add('MUL', r'\*')
lg.add('DIV', r'\/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')
lg.add('OPEN_COL', r'\[')
lg.add('CLOSE_COL', r'\]')

lg.add('INT', r'inteiro')
lg.add('STRING', r'palavra')
lg.add('ELSE', r'senao')
lg.add('IF', r'se')
lg.add('WHILE', r'enquanto')
lg.add('FOR', r'para')

lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

In [2]:
tokens=lexer.lex('inteiro a; palavra b; a = 10; b = teste;')
list(tokens)

[Token('INT', 'inteiro'),
 Token('ID', 'a'),
 Token('SEMICOL', ';'),
 Token('STRING', 'palavra'),
 Token('ID', 'b'),
 Token('SEMICOL', ';'),
 Token('ID', 'a'),
 Token('EQUALS', '='),
 Token('NUMBER', '10'),
 Token('SEMICOL', ';'),
 Token('ID', 'b'),
 Token('EQUALS', '='),
 Token('ID', 'teste'),
 Token('SEMICOL', ';')]

In [3]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp,size):
        self.id = id
        self.tp = tp
        self.size = size


    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class For(BaseBox):
    def __init__(self, idinic, exprinic, expr1, comp, expr2, idincr, exprincr, ie1):
        self.idinic=idinic
        self.exprinic=exprinic
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.idincr=idincr
        self.exprincr=exprincr
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_for(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value, index):
        self.value = value
        self.index = index

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


In [4]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS', 'OPEN_COL', 'CLOSE_COL',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'STRING', 'ID','SEMICOL',
     'EQUALS','COMP','IF','ELSE','WHILE','FOR'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : STRING ID SEMICOL')
def vardecl_string(p):
    return VarDecl(p[1].getstr(), "string", None)

@pg.production('vardecl : INT ID SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int", None)

@pg.production('vardecl : INT ID OPEN_COL NUMBER CLOSE_COL SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int", p[3].getstr())


##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('openstatement : FOR OPEN_PARENS ID EQUALS expression SEMICOL expression COMP expression SEMICOL ID EQUALS expression CLOSE_PARENS openstatement')
def statement_while(p):
    return For (p[2].getstr(),p[4],p[6],p[7],p[8],p[10].getstr(),p[12],p[14])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('closedstatement : FOR OPEN_PARENS ID EQUALS expression SEMICOL expression COMP expression SEMICOL ID EQUALS expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return For (p[2].getstr(),p[4],p[6],p[7],p[8],p[10].getstr(),p[12],p[14])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr(),None)

@pg.production('expression : ID OPEN_COL NUMBER CLOSE_COL')
def expression_id(p):
    return Id(p[0].getstr(),p[2].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [5]:
with open("arquivo.pbr", "r") as arquivo:
    arvore=parser.parse(lexer.lex(arquivo.read()))
arvore

In [6]:
# VISITOR PARA TABELA DE SÍMBOLOS

ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp


In [7]:
#VISITOR PARA DECORAR TIPOS

class Decorator(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)

    def visit_for(self, i):
        if i.idinic in ST:
          i.inicdecor_type=ST[i.idinic]
        else:
          raise AssertionError('id not declared')
        i.exprinic.accept(self)
        i.expr1.accept(self)
        i.expr2.accept(self)
        if i.idincr in ST:
          i.incrdecor_type=ST[i.idincr]
        else:
          raise AssertionError('id not declared')
        i.exprincr.accept(self)
        i.ie1.accept(self)


    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id not declared')


    def visit_number(self, i):
        i.decor_type="int" if type(i.value) is int else "string"


    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        elif a.left.decor_type=="string" and a.right.decor_type=="string":
          a.decor_type="string"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        elif a.left.decor_type=="string" and a.right.decor_type=="string":
          a.decor_type="string"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        elif a.left.decor_type=="string" and a.right.decor_type=="string":
          a.decor_type="string"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        elif a.left.decor_type=="string" and a.right.decor_type=="string":
          a.decor_type="string"



In [8]:
# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('type error')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_for(self, i):
        # verificação da inicialização do for
        if i.inicdecor_type!=i.exprinic.decor_type:
          raise AssertionError('type error')

        # verificação da comaração do for
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

        # verificação da atualização do for
        if i.incrdecor_type!=i.exprincr.decor_type:
          raise AssertionError('type error')


In [9]:
class IntermediateCode(Visitor):

    def __init__(self):
        self.level = 0  # nível do if-then-else
        self.loop = 0  # número do loop
        self.comp = {"==": "8", "!=": "9", "<": "10", "<=": "11", ">": "12", ">=": "13"}  # códigos dos comparadores
        self.instructions = []  # Lista para armazenar as instruções

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts is not None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        i.expr.accept(self)
        self.instructions.append(f"sto 0 {i.id}")

    def visit_ifelse(self, i):
        i.expr2.accept(self)
        i.expr1.accept(self)
        self.instructions.append(f"opr 0 {self.comp[i.comp.getstr()]}")
        self.level += 1
        self.instructions.append(f"jpc 0 true{self.level}")
        self.instructions.append(f"jmp 0 false{self.level}")
        self.instructions.append(f"true{self.level}:")
        i.ie1.accept(self)
        self.instructions.append(f"jmp final{self.level}")
        self.instructions.append(f"false{self.level}:")
        if i.ie2 is not None:
            i.ie2.accept(self)
        self.instructions.append(f"final{self.level}:")
        self.level -= 1

    def visit_while(self, i):
        self.loop += 1
        self.instructions.append(f"loop{self.loop}:")
        i.expr2.accept(self)
        i.expr1.accept(self)
        self.instructions.append(f"opr 0 {self.comp[i.comp.getstr()]}")
        self.level += 1
        self.instructions.append(f"jpc 0 true{self.level}")
        self.instructions.append(f"jmp 0 final{self.level}")
        self.instructions.append(f"true{self.level}:")
        i.ie1.accept(self)
        self.instructions.append(f"jmp loop{self.loop}")
        self.instructions.append(f"final{self.level}:")
        self.level -= 1

    def visit_for(self, i):
        self.loop += 1
        i.exprinic.accept(self)
        self.instructions.append(f"sto 0 {i.idinic}")
        self.instructions.append(f"loop{self.loop}:")
        i.expr2.accept(self)
        i.expr1.accept(self)
        self.instructions.append(f"opr 0 {self.comp[i.comp.getstr()]}")
        self.level += 1
        self.instructions.append(f"jpc 0 true{self.level}")
        self.instructions.append(f"jmp 0 final{self.level}")
        self.instructions.append(f"true{self.level}:")
        i.ie1.accept(self)
        i.exprincr.accept(self)
        self.instructions.append(f"sto 0 {i.idincr}")
        self.instructions.append(f"jmp loop{self.loop}")
        self.instructions.append(f"final{self.level}:")
        self.level -= 1

    def visit_number(self, i):
        self.instructions.append(f"lit 0 {i.value}")

    def visit_id(self, i):
        if i.index is None:
            self.instructions.append(f"lod 0 {i.value}")
        else:
            self.instructions.append(f"lod 0 {i.value}[{i.index}]")

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        self.instructions.append("opr 0 2")

    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        self.instructions.append("opr 0 3")

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        self.instructions.append("opr 0 4")

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        self.instructions.append("opr 0 5")


In [10]:
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
intermediate_code = IntermediateCode()
arvore.accept(intermediate_code)
for instruction in intermediate_code.instructions:
    print(instruction)

lit 0 5
sto 0 a
lit 0 2
sto 0 b
lod 0 b
lod 0 a
opr 0 12
jpc 0 true1
jmp 0 false1
true1:
lod 0 a
lod 0 b
opr 0 2
sto 0 a
jmp final1
false1:
lod 0 a
lod 0 b
opr 0 2
sto 0 b
final1:


In [11]:
class SimpleInterpreter:
    def __init__(self, instructions):
        self.instructions = instructions
        self.stack = []
        self.variables = {}

    def run(self):
        for instr in self.instructions:
            self.process_instruction(instr)
            print(f"Pilha atual: {self.stack}")

    def process_instruction(self, instr):
        parts = instr.split()
        opcode = parts[0]

        if opcode == "lit":
            value = int(parts[2])
            self.stack.append(value)
            print(f"Carrega literal {value} na pilha")

        elif opcode == "sto":
            var_name = parts[2]
            self.variables[var_name] = self.stack.pop()
            print(f"Atribui o topo da pilha à variável {var_name}")

        elif opcode == "lod":
            var_name = parts[2]
            value = self.variables.get(var_name, 0)
            self.stack.append(value)
            print(f"Carrega a variável {var_name} com valor {value} na pilha")

        elif opcode == "opr":
            operator = int(parts[2])
            self.perform_operation(operator)

    def perform_operation(self, operator):
        if operator == 2:  # Soma
            a, b = self.stack.pop(), self.stack.pop()
            self.stack.append(a + b)
            print(f"Realiza soma: {a} + {b}")

        elif operator == 3:  # Subtração
            a, b = self.stack.pop(), self.stack.pop()
            self.stack.append(b - a)
            print(f"Realiza subtração: {b} - {a}")

        elif operator == 4:  # Multiplicação
            a, b = self.stack.pop(), self.stack.pop()
            self.stack.append(a * b)
            print(f"Realiza multiplicação: {a} * {b}")

        elif operator == 5:  # Divisão
            a, b = self.stack.pop(), self.stack.pop()
            self.stack.append(b // a)
            print(f"Realiza divisão: {b} // {a}")


interpreter = SimpleInterpreter(intermediate_code.instructions)
interpreter.run()


Carrega literal 5 na pilha
Pilha atual: [5]
Atribui o topo da pilha à variável a
Pilha atual: []
Carrega literal 2 na pilha
Pilha atual: [2]
Atribui o topo da pilha à variável b
Pilha atual: []
Carrega a variável b com valor 2 na pilha
Pilha atual: [2]
Carrega a variável a com valor 5 na pilha
Pilha atual: [2, 5]
Pilha atual: [2, 5]
Pilha atual: [2, 5]
Pilha atual: [2, 5]
Pilha atual: [2, 5]
Carrega a variável a com valor 5 na pilha
Pilha atual: [2, 5, 5]
Carrega a variável b com valor 2 na pilha
Pilha atual: [2, 5, 5, 2]
Realiza soma: 2 + 5
Pilha atual: [2, 5, 7]
Atribui o topo da pilha à variável a
Pilha atual: [2, 5]
Pilha atual: [2, 5]
Pilha atual: [2, 5]
Carrega a variável a com valor 7 na pilha
Pilha atual: [2, 5, 7]
Carrega a variável b com valor 2 na pilha
Pilha atual: [2, 5, 7, 2]
Realiza soma: 2 + 7
Pilha atual: [2, 5, 9]
Atribui o topo da pilha à variável b
Pilha atual: [2, 5]
Pilha atual: [2, 5]


In [12]:

from rply import LexerGenerator, ParserGenerator
from rply.errors import LexingError, ParsingError


def execute_intermediate_code(instructions):
    interpreter = SimpleInterpreter(instructions)
    interpreter.run()

def main():
    code = """
    inteiro a;
    inteiro b;

    a = 1;
    b = 2;

    se (a > b)
        a = a + b;
    senao
        b = a + b;
    """

    try:
        tokens = lexer.lex(code)
        parse_tree = parser.parse(tokens)
        symbol_table_visitor = SymbolTable()
        symbol_table_visitor.visit_prog(parse_tree)

        decorator_visitor = Decorator()
        decorator_visitor.visit_prog(parse_tree)

        type_verifier = TypeVerifier()
        type_verifier.visit_prog(parse_tree)

        intermediate_code_visitor = IntermediateCode()
        intermediate_code_visitor.visit_prog(parse_tree)

        execute_intermediate_code(intermediate_code_visitor.instructions)

    except LexingError as e:
        print(f"Erro de lexing: {e}")
    except ParsingError as e:
        print(f"Erro de parsing: {e}")
    except AssertionError as e:
        print(f"Erro de execução: {e}")

if __name__ == "__main__":
    main()

Carrega literal 1 na pilha
Pilha atual: [1]
Atribui o topo da pilha à variável a
Pilha atual: []
Carrega literal 2 na pilha
Pilha atual: [2]
Atribui o topo da pilha à variável b
Pilha atual: []
Carrega a variável b com valor 2 na pilha
Pilha atual: [2]
Carrega a variável a com valor 1 na pilha
Pilha atual: [2, 1]
Pilha atual: [2, 1]
Pilha atual: [2, 1]
Pilha atual: [2, 1]
Pilha atual: [2, 1]
Carrega a variável a com valor 1 na pilha
Pilha atual: [2, 1, 1]
Carrega a variável b com valor 2 na pilha
Pilha atual: [2, 1, 1, 2]
Realiza soma: 2 + 1
Pilha atual: [2, 1, 3]
Atribui o topo da pilha à variável a
Pilha atual: [2, 1]
Pilha atual: [2, 1]
Pilha atual: [2, 1]
Carrega a variável a com valor 3 na pilha
Pilha atual: [2, 1, 3]
Carrega a variável b com valor 2 na pilha
Pilha atual: [2, 1, 3, 2]
Realiza soma: 2 + 3
Pilha atual: [2, 1, 5]
Atribui o topo da pilha à variável b
Pilha atual: [2, 1]
Pilha atual: [2, 1]
